## **Project**: Natural products from the Palaeolithic

## **Section**: Genome assemblies

Anan Ibrahim, 01.01.2022

**Contents**
 - **Step1**: Create conda envirorment with required dependencies if not already installed 
 - **Step2**: Download sequencing results from eurofins using commandline
 - **Step3**: Genome assembly

##########

**Step1**: Create conda envirorment with required dependencies if not already installed 

##########

In [ ]:
# All conda envs can be found in EMN001_Paleofuran/02-scripts/ENVS_*.yml
conda env create -f plasmid_assembly.yml
conda env create -f single_genome_assembly.yml

##########

**Step2**: Download sequencing results from eurofins using commandline

##########

In [ ]:
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly && cd "$_"
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input && cd "$_"
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Output && cd "$_"
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Scripts && cd "$_"

**PROJECT SPECIFIC**

*Manually:* Add the fastq sequences of the genomesretrieved from eurofins (in the downloads folder below) in folders named by samples

In [ ]:
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_protegens && cd "$_"

In [ ]:
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_temperata && cd "$_"

In [ ]:
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/eurofins-downloads && cd "$_"

wget -m --ftp-user=########### --ftp-password=######### ftp://ftp.gatc-biotech.com/2021-12-21/

*Manually:* Add the ref sequences of the genomes retrieved from eurofins in folders named by samples

In [ ]:
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_protegens_ref_seq && cd "$_"

In [ ]:
mkdir /Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_temperata_ref_seq && cd "$_"

How to deal with jupyter notebook on your local computer

 - If miniconda is not installed, install miniconda in your local directory. Close and reopen the terminal

 - Now create en env bytyping in the terminal: conda env create -n jupyter-notebook -c anaconda jupyter
 
 - To modify the jupyter notebook after downloading the recent copy of the JN from file-zilla: 
 
 - Activate the env: conda activate jupyter-notebook
 
 - Run by typing: jupyter-notebook

##########

**Step3**: Genome assembly

##########

*NOTE:* Before running please make sure you change the IN OUT REF directories paths according to the project.

*NOTE:* Before running please make sure you change the the file names in the IN OUT directory to always match *_3_1.fastq.gz and *_3_2.fastq.gz

### (a) **Protegens**

In [ ]:
#!/bin/bash
eval "$(conda shell.bash hook)"

# Always remember to put all seq in folders named according to the samples
# "/Net/Groups/ccdata/projects/ancientDNA/Plasmid-assembly/Input/XXX_batch/" 
# Always remember to put all seq in
# "/Net/Groups/ccdata/projects/ancientDNA/Plasmid-assembly/Input/XXX_batch_ref_seq/" 
# before running the script

IN=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_protegens #change the names accordingly 
OUT=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Output/p_protegens #change the names accordingly
REF=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_protegens_ref_seq #change the names accordingly
LP=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/landing_pad

mkdir $OUT

#############################################################################################
#Read filtering
#############################################################################################

conda activate plasmid_assembly

# (1) Run FASTQC on raw data
mkdir  $OUT/01_FastQC_results
for F in $IN/*; do 
N=$(basename $F) ;
mkdir $OUT/01_FastQC_results/$N ;
cd "$F"; 
fastqc $F/*_1.fastq.gz $F/*_2.fastq.gz -t 30 -o $OUT/01_FastQC_results/$N;
done 

# (2) Run trimmomatic
mkdir  $OUT/02_Trimmomatic_results 
for F in $IN/*; do
N=$(basename $F) ;
mkdir $OUT/02_Trimmomatic_results/$N; 
cd "$F";
trimmomatic PE -threads 30 \
-trimlog $N.trimlog.txt \
-summary $N.stats.txt \
$F/*_1.fastq.gz $F/*_2.fastq.gz \
-baseout $N.filtered100.fastq.gz \
MINLEN:100 SLIDINGWINDOW:9:35 ; 
mv *.txt $OUT/02_Trimmomatic_results/$N
mv *_1P.fastq.gz $OUT/02_Trimmomatic_results/$N
mv *_2P.fastq.gz $OUT/02_Trimmomatic_results/$N
mv *_1U.fastq.gz $OUT/02_Trimmomatic_results/$N
mv *_2U.fastq.gz $OUT/02_Trimmomatic_results/$N
done 

# (3) Run FastQC again 
mkdir $OUT/03_FastQC_results_post_trim
for F in $OUT/02_Trimmomatic_results/*; do 
N=$(basename $F) ;
mkdir $OUT/03_FastQC_results_post_trim/$N ;
cd "$F"; 
fastqc $F/*_1P.fastq.gz $F/*_2P.fastq.gz -t 30 -o $OUT/03_FastQC_results_post_trim/$N;
done

#############################################################################################
#Denovo genome assembly
#############################################################################################

conda activate single_genome_assembly

mkdir $OUT/06_spades_genome_assembly
for F in $OUT/02_Trimmomatic_results/*; do
N=$(basename $F) ;
mkdir $OUT/06_spades_genome_assembly/$N && cd "$_"
spades.py --isolate --careful -t 29 \
-1 $OUT/02_Trimmomatic_results/$N/*_1P.fastq.gz \
-2 $OUT/02_Trimmomatic_results/$N/*_2P.fastq.gz \
-o $OUT/06_spades_genome_assembly/$N
seqkit fx2tab $OUT/06_spades_genome_assembly/$N/scaffolds.fasta | csvtk mutate -H -t -f 1 -p "length_([0-9]+)" | awk -F "\t" '$4>=1000' | seqkit tab2fx > filtered_scaffolds.fasta
; done

#############################################################################################
#Denovo genome assembly::Which Reference genome was it? and where is the LAnding pad?
#############################################################################################

conda activate plasmid_assembly

mkdir  $OUT/05_Minimap2_results_for_denovo5
for F in $OUT/05_spades_genome_assembly/*; do 
N=$(basename $F) ;
mkdir $OUT/05_Minimap2_results_for_denovo5/$N && cd "$_"
minimap2 -ax asm5 $REF/*_dgacA.fasta $OUT/05_spades_genome_assembly/$N/filtered_scaffolds.fasta > aln_ref_genome.sam 
minimap2 -ax asm5 $LP/*.fna $OUT/05_spades_genome_assembly/$N/filtered_scaffolds.fasta > aln_LP_orig.sam
minimap2 -ax asm5 $LP/*.fasta $OUT/05_spades_genome_assembly/$N/filtered_scaffolds.fasta > aln_LP_2BGC_genes.sam
#minimap2 -x map-ont -d $LP/originalLP.mmi $LP/*.fna
#minimap2 -a $LP/originalLP.mmi $OUT/05_spades_genome_assembly/$N/scaffolds.fasta > test.sam;
done


#############################################################################################
#Denovo genome assembly::Which Reference genome is the genomes?
#############################################################################################

conda activate taxonomic-binning

mkdir $OUT/06_MMseqs2_alignment5
for F in $OUT/05_spades_genome_assembly/*; do 
N=$(basename $F) ;
mkdir $OUT/06_MMseqs2_alignment5/$N && cd "$_"; 
mmseqs createdb $REF/*_dgacA.fasta ref
#linearise the reference sequence when their is contigs 
cat $OUT/05_spades_genome_assembly/$N/filtered_scaffolds.fasta | sed -e '1!{/^>.*/d;}' | sed  ':a;N;$!ba;s/\n//2g' > scaffold.fna
mmseqs createdb scaffold.fna scaffold
mmseqs prefilter scaffold ref sc_pref 
mmseqs align scaffold ref sc_pref sc_aln -a --max-seqs 10000
mmseqs createtsv scaffold ref sc_aln sc_aln.tsv
mmseqs convertalis scaffold ref --format-output "query,target,evalue,qaln,taln,pident,nident,mismatch" sc_aln sc_aln.m8 --search-type 3;
done

#############################################################################################
#Ref.based genome alignment
#############################################################################################
# dgcaA # 

conda activate plasmid_assembly

# Run minimap2 and samtools (reference based alignment) ::: check reference
mkdir  $OUT/05_Minimap2_samtools_ref_alignment
for F in $OUT/02_Trimmomatic_results/*; do 
N=$(basename $F) ;
mkdir  $OUT/05_Minimap2_samtools_ref_alignment/$N;
cd $OUT/05_Minimap2_samtools_ref_alignment/$N;
minimap2 -ax sr $REF/*_dgacA.fasta \
$OUT/02_Trimmomatic_results/$N/*_1P.fastq.gz \
$OUT/02_Trimmomatic_results/$N/*_2P.fastq.gz > $N.minimap2.sam; done

conda activate samtools

for F in $OUT/05_Minimap2_samtools_ref_alignment/*; do 
N=$(basename $F);
cd $OUT/05_Minimap2_samtools_ref_alignment/$N;
samtools view -S -b *.sam > aln.bam # Convert sam to bam
samtools sort aln.bam -o aln_sorted.bam --threads 28 # Sort the alignment
# Get consensus fastq file
samtools mpileup -uf $REF/*_dgacA.fasta aln_sorted.bam | bcftools call -c | vcfutils.pl vcf2fq > $N.consensus.fastq
# Convert .fastq to .fasta
seqtk seq -aQ64 -q13 -n N $N.consensus.fastq > $N.fasta; done


# Map the reads to the p.protegens refs (x3)
conda activate plasmid_assembly

# Run Minimap2 allignment (for all three)
mkdir  $OUT/05_Minimap2_results_for_denovo5
for F in $OUT/06_spades_genome_assembly/*; do 
N=$(basename $F) ;
mkdir  $OUT/05_Minimap2_results_for_denovo5/$N && cd "$_";
minimap2 -ax asm20 $REF/*_dgacA.fasta $OUT/06_spades_genome_assembly/$N/contigs.fasta > aln.sam ;
done


### (a) **Temperata**

In [ ]:
#!/bin/bash
eval "$(conda shell.bash hook)"

# Always remember to put all seq in folders named according to the samples
# "/Net/Groups/ccdata/projects/ancientDNA/Plasmid-assembly/Input/XXX_batch/" 
# Always remember to put all seq in
# "/Net/Groups/ccdata/projects/ancientDNA/Plasmid-assembly/Input/XXX_batch_ref_seq/" 
# before running the script

IN=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_temperata #change the names accordingly 
OUT=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Output/p_temperata #change the names accordingly
REF=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/p_temperata_ref_seq #change the names accordingly
LP=/Net/Groups/ccdata/projects/ancientDNA/Single_genome_assembly/Input/landing_pad

mkdir $OUT

#############################################################################################
#Read filtering
#############################################################################################

conda activate plasmid_assembly

# (1) Run FASTQC on raw data
mkdir  $OUT/01_FastQC_results
for F in $IN/*; do 
N=$(basename $F) ;
mkdir $OUT/01_FastQC_results/$N ;
cd "$F"; 
fastqc $F/*_1.fastq.gz $F/*_2.fastq.gz -t 30 -o $OUT/01_FastQC_results/$N;
done 

# (2) Run trimmomatic
mkdir  $OUT/02_Trimmomatic_results 
for F in $IN/*; do
N=$(basename $F) ;
mkdir $OUT/02_Trimmomatic_results/$N; 
cd "$F";
trimmomatic PE -threads 30 \
-trimlog $N.trimlog.txt \
-summary $N.stats.txt \
$F/*_1.fastq.gz $F/*_2.fastq.gz \
-baseout $N.filtered100.fastq.gz \
MINLEN:100 SLIDINGWINDOW:9:35 ; 
mv *.txt $OUT/02_Trimmomatic_results/$N
mv *_1P.fastq.gz $OUT/02_Trimmomatic_results/$N
mv *_2P.fastq.gz $OUT/02_Trimmomatic_results/$N
mv *_1U.fastq.gz $OUT/02_Trimmomatic_results/$N
mv *_2U.fastq.gz $OUT/02_Trimmomatic_results/$N
done 

# (3) Run FastQC again 
mkdir $OUT/03_FastQC_results_post_trim
for F in $OUT/02_Trimmomatic_results/*; do 
N=$(basename $F) ;
mkdir $OUT/03_FastQC_results_post_trim/$N ;
cd "$F"; 
fastqc $F/*_1P.fastq.gz $F/*_2P.fastq.gz -t 30 -o $OUT/03_FastQC_results_post_trim/$N;
done

#############################################################################################
#Denovo genome assembly
#############################################################################################

conda activate single_genome_assembly

mkdir $OUT/05_spades_genome_assembly
for F in $OUT/02_Trimmomatic_results/*; do
N=$(basename $F) ;
mkdir $OUT/05_spades_genome_assembly/$N && cd "$_"
spades.py -k 21,33,55,77 --only-assembler --careful -t 29 \
-1 $OUT/02_Trimmomatic_results/$N/*_1P.fastq.gz \
-2 $OUT/02_Trimmomatic_results/$N/*_2P.fastq.gz \
-o $OUT/05_spades_genome_assembly/$N
seqkit fx2tab $OUT/05_spades_genome_assembly/$N/scaffolds.fasta | csvtk mutate -H -t -f 1 -p "length_([0-9]+)" | awk -F "\t" '$4>=1000' | seqkit tab2fx > filtered_scaffolds.fasta;
done


#############################################################################################
#Denovo genome assembly::Which Reference genome is the genomes?
#############################################################################################

conda activate taxonomic-binning

mkdir $OUT/06_MMseqs2_alignment5
for F in $OUT/05_spades_genome_assembly/*; do 
N=$(basename $F) ;
mkdir $OUT/06_MMseqs2_alignment5/$N && cd "$_";
mmseqs createdb $REF/*.fna ref
#linearise the reference sequence when their is contigs 
cat $OUT/05_spades_genome_assembly/$N/filtered_scaffolds.fasta | sed -e '1!{/^>.*/d;}' | sed  ':a;N;$!ba;s/\n//2g' > scaffold.fna
mmseqs createdb scaffold.fna scaffold
mmseqs prefilter scaffold ref sc_pref 
mmseqs align scaffold ref sc_pref sc_aln -a --max-seqs 10000
mmseqs createtsv scaffold ref sc_aln sc_aln.tsv
mmseqs convertalis scaffold ref --format-output "query,target,evalue,qaln,taln,pident,nident,mismatch" sc_aln sc_aln.m8 --search-type 3;
done

#############################################################################################
#Ref.based genome alignment
#############################################################################################
# dgcaA # 

conda activate plasmid_assembly

# Run minimap2 and samtools (reference based alignment) ::: check reference
mkdir  $OUT/05_Minimap2_samtools_ref_alignment
for F in $OUT/02_Trimmomatic_results/*; do 
N=$(basename $F) ;
mkdir  $OUT/05_Minimap2_samtools_ref_alignment/$N;
cd $OUT/05_Minimap2_samtools_ref_alignment/$N;
minimap2 -ax sr $REF/*_dgacA.fasta \
$OUT/02_Trimmomatic_results/$N/*_1P.fastq.gz \
$OUT/02_Trimmomatic_results/$N/*_2P.fastq.gz > $N.minimap2.sam; done

conda activate samtools

for F in $OUT/05_Minimap2_samtools_ref_alignment/*; do 
N=$(basename $F);
cd $OUT/05_Minimap2_samtools_ref_alignment/$N;
samtools view -S -b *.sam > aln.bam # Convert sam to bam
samtools sort aln.bam -o aln_sorted.bam --threads 28 # Sort the alignment
# Get consensus fastq file
samtools mpileup -uf $REF/*_dgacA.fasta aln_sorted.bam | bcftools call -c | vcfutils.pl vcf2fq > $N.consensus.fastq
# Convert .fastq to .fasta
seqtk seq -aQ64 -q13 -n N $N.consensus.fastq > $N.fasta; done


# Map the reads to the p.protegens refs (x3)
conda activate plasmid_assembly

# Run Minimap2 allignment (for all three)
mkdir  $OUT/05_Minimap2_results_for_denovo5
for F in $OUT/06_spades_genome_assembly/*; do 
N=$(basename $F) ;
mkdir  $OUT/05_Minimap2_results_for_denovo5/$N && cd "$_";
minimap2 -ax asm20 $REF/*_dgacA.fasta $OUT/06_spades_genome_assembly/$N/contigs.fasta > aln.sam ;
done
